In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim

# 데이터 읽기 및 전처리
data = pd.read_csv("car_evaluation.csv", header=None)
columns = ["buying", "maint", "doors", "persons", "lug_boot", "safety", "class"]
data.columns = columns

# 라벨 인코딩
label_encoder = {}
for column in columns:
    label_encoder[column] = LabelEncoder()
    data[column] = label_encoder[column].fit_transform(data[column])

# 독립 변수와 종속 변수 분리
X = data.drop("class", axis=1)
y = data["class"]

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 데이터 정규화
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 텐서 변환
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.LongTensor(y_train.values)
X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.LongTensor(y_test.values)

# 분류 모델 정의
class ClassificationModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(ClassificationModel, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)
    
    def forward(self, x):
        return self.fc(x)

# 모델 초기화
input_dim = X_train_tensor.shape[1]
output_dim = len(label_encoder["class"].classes_)
model = ClassificationModel(input_dim, output_dim)

# 손실 함수와 옵티마이저 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# 모델 학습
epochs = 100
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

# 평가
model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor)
    test_loss = criterion(outputs, y_test_tensor)
    predicted_classes = torch.argmax(outputs, dim=1)
    accuracy = (predicted_classes == y_test_tensor).float().mean()

    print(f"Test Loss: {test_loss.item():.4f}")
    print(f"Accuracy: {accuracy.item():.4f}")


Epoch [10/100], Loss: 1.1622
Epoch [20/100], Loss: 1.0354
Epoch [30/100], Loss: 0.9509
Epoch [40/100], Loss: 0.8936
Epoch [50/100], Loss: 0.8534
Epoch [60/100], Loss: 0.8238
Epoch [70/100], Loss: 0.8014
Epoch [80/100], Loss: 0.7839
Epoch [90/100], Loss: 0.7699
Epoch [100/100], Loss: 0.7584
Test Loss: 0.7878
Accuracy: 0.6590
